In [11]:
# Импортируем библиотеку pandas, которая будет использоваться для работы с данными
import pandas as pd

# Читаем данные из файла "dataset.csv" и сохраняем их в переменную data
data = pd.read_csv("dataset.csv")

# Выводим первые 5 строк датафрейма для ознакомления с данными
print(data.head())

   clientbankpartner_pin  client_pin  partner_src_type_ccode  \
0                 122027        5579                       4   
1                 270277        5585                       4   
2                 238679        5586                       4   
3                 118398        5587                       4   
4                  10402        5588                       1   

  client_start_date partnerrolestart_date  
0        2019-06-25            2019-02-01  
1        2020-05-07            2020-04-01  
2        2020-02-27            2020-02-01  
3        2020-03-26            2019-09-01  
4        2019-04-05            2019-01-01  


In [16]:
# Преобразуем столбцы с датами в формат datetime
data['client_start_date'] = pd.to_datetime(data['client_start_date'])
data['partnerrolestart_date'] = pd.to_datetime(data['partnerrolestart_date'])

# Определяем контрольные даты для разделения данных
cutoff_dates = {
    'train_start': pd.to_datetime("2020-03-01"),
    'train_end': pd.to_datetime("2020-06-01"),
    'test_end': pd.to_datetime("2020-09-01"),
    'target': pd.to_datetime("2020-12-01")
}

# Функция для фильтрации данных по заданным датам
def filter_data(start_date, end_date=None):
    """
    Фильтрует данные по заданным датам.
    :param start_date: начальная дата для фильтрации
    :param end_date: конечная дата для фильтрации (необязательная)
    :return: отфильтрованный DataFrame
    """
    if end_date:
        return data[(data['client_start_date'] >= start_date) & (data['client_start_date'] < end_date)]
    else:
        return data[data['client_start_date'] >= start_date]

# Разделяем данные на train, test и target
train_data = filter_data(cutoff_dates['train_start'], cutoff_dates['train_end'])
test_data = filter_data(cutoff_dates['train_end'], cutoff_dates['test_end'])
target_data = filter_data(cutoff_dates['test_end'])

"""
Мы использовали подход, при котором данные разделяются на равные части по 3 месяца.
Это позволяет равномерно распределить данные для обучения, тестирования и целевого прогнозирования,
что важно для обеспечения стабильности и точности модели на различных временных интервалах.
"""

# Функция для агрегирования данных
def aggregate_data(df):
    """
    Агрегирует данные по 'clientbankpartner_pin', вычисляя максимальные значения и количество клиентов.
    :param df: исходный DataFrame
    :return: агрегированный DataFrame
    """
    return df.groupby('clientbankpartner_pin').agg({
        'partner_src_type_ccode': 'first',
        'client_start_date': 'max',
        'partnerrolestart_date': 'max',
        'client_pin': 'count'
    }).reset_index().rename(columns={'client_pin': 'num_clients'})

# Применяем агрегацию к train, test и target данным
train_data = aggregate_data(train_data)
test_data = aggregate_data(test_data)
target_data = aggregate_data(target_data)

# Выводим первые 5 строк для проверки
print("Train Data:")
print(train_data.head())

print("\nTest Data:")
print(test_data.head())

print("\nTarget Data:")
print(target_data.head())

Train Data:
   clientbankpartner_pin  partner_src_type_ccode client_start_date  \
0                      1                       5        2020-05-29   
1                     30                       5        2020-04-12   
2                     35                       5        2020-05-22   
3                     38                       5        2020-04-18   
4                     40                       5        2020-05-22   

  partnerrolestart_date  num_clients  
0            2019-08-01            1  
1            2019-11-01            1  
2            2018-12-01            3  
3            2020-03-01            1  
4            2019-07-01            1  

Test Data:
   clientbankpartner_pin  partner_src_type_ccode client_start_date  \
0                      1                       5        2020-06-05   
1                      2                       5        2020-06-30   
2                     10                       5        2020-06-30   
3                     17                 

In [17]:
# Импортируем библиотеку numpy для работы с числовыми данными
import numpy as np

# Устанавливаем пороговое значение для количества клиентов на основе 99-го процентиля: это мы используем для удаления выбросов, т.к. при просмотре датасета выяснилось, что один из клиентов привлек более 9000 партнеров
threshold_num_clients = np.percentile(train_data['num_clients'], 99)

# Фильтруем train_data, удаляя строки с количеством клиентов выше порогового значения
train_data = train_data[train_data['num_clients'] <= threshold_num_clients]

# Функция для подсчета недавних клиентов (за последний месяц)
def count_recent_clients(data, cutoff_date):
    one_month_ago = cutoff_date - pd.DateOffset(months=1)
    recent_clients = data.groupby('clientbankpartner_pin')['client_start_date'].transform(
        lambda dates: sum(dates > one_month_ago))
    return recent_clients

# Функция для подсчета остальных клиентов (старше одного месяца)
def count_other_clients(data, cutoff_date):
    one_month_ago = cutoff_date - pd.DateOffset(months=1)
    other_clients = data.groupby('clientbankpartner_pin')['client_start_date'].transform(
        lambda dates: sum(dates < one_month_ago))
    return other_clients

# Функция для вычисления среднего времени между появлениями клиентов
def avg_time_between_clients(group):
    if len(group) > 1:
        return (group['client_start_date'].max() - group['client_start_date'].min()).days / (len(group) - 1)
    else:
        return 0

# Функция для добавления новых столбцов в данные
def add_calculated_columns(data, cutoff_date):
    return data.assign(
        days_since_last_client=(cutoff_date - data['client_start_date']).dt.days // 1,
        days_since_partner_start=(cutoff_date - data['partnerrolestart_date']).dt.days // 1,
        num_recent_clients=count_recent_clients(data, cutoff_date),
        num_other_clients=count_other_clients(data, cutoff_date),
        avg_time_between_clients=(data['client_start_date'].max() - data['client_start_date']).dt.days // data['num_clients'],
        time_as_partner=(cutoff_date - data['partnerrolestart_date']).dt.days
    )

# Добавляем новые столбцы в train, test и target данных
train_data = add_calculated_columns(train_data, cutoff_dates['train_end'])
test_data = add_calculated_columns(test_data, cutoff_dates['test_end'])
target_data = add_calculated_columns(target_data, cutoff_dates['target'])

# Определяем множество клиентов в train, test и target данных
train_clients = set(train_data['clientbankpartner_pin'])
new_clients_in_test = set(test_data['clientbankpartner_pin'])
train_churn = train_data['clientbankpartner_pin'].apply(lambda pin: pin not in new_clients_in_test)
test_clients = set(test_data['clientbankpartner_pin'])
new_clients_in_target = set(target_data['clientbankpartner_pin'])
test_churn = test_data['clientbankpartner_pin'].apply(lambda pin: pin not in new_clients_in_target)

# Определяем список признаков для анализа
features = [
    'clientbankpartner_pin',
    'days_since_last_client',
    'days_since_partner_start',
    'num_clients',
    'num_recent_clients',
    'avg_time_between_clients',
    'time_as_partner',
    'partner_src_type_ccode',
    'num_other_clients'
]

# Оставляем только необходимые признаки в train, test и target данных
train_data = train_data[features]
test_data = test_data[features]
target_data = target_data[features]

# Добавляем столбец 'churn' для train и test данных
train_data['churn'] = train_churn.astype(int)
test_data['churn'] = test_churn.astype(int)


In [18]:
# Разделяем данные на признаки (X) и целевую переменную (y) для обучения
X_train = train_data.drop(columns=['clientbankpartner_pin', 'churn'])
y_train = train_data['churn']

# Разделяем данные на признаки (X) и целевую переменную (y) для тестирования
X_test = test_data.drop(columns=['clientbankpartner_pin', 'churn'])
y_test = test_data['churn']

# Разделяем целевые данные на признаки (X) без целевой переменной
X_target = target_data.drop(columns=['clientbankpartner_pin'])


In [19]:
# Импортируем StandardScaler для стандартизации числовых признаков
from sklearn.preprocessing import StandardScaler

# Создаем экземпляр StandardScaler
scaler = StandardScaler()

# Копируем тренировочные данные для дальнейшей обработки
X_train_scaled = X_train.copy()

# Определяем числовые признаки, которые будут стандартизированы
num_features = [
    'days_since_last_client',
    'days_since_partner_start',
    'num_clients',
    'num_recent_clients',
    'avg_time_between_clients',
    'time_as_partner',
    'num_other_clients'
]

# Стандартизируем числовые признаки в тренировочных данных
X_train_scaled[num_features] = scaler.fit_transform(X_train[num_features])

# Копируем тестовые данные для дальнейшей обработки
X_test_scaled = X_test.copy()

# Стандартизируем числовые признаки в тестовых данных
X_test_scaled[num_features] = scaler.transform(X_test[num_features])

# Копируем целевые данные для дальнейшей обработки
X_target_scaled = X_target.copy()

# Стандартизируем числовые признаки в целевых данных
X_target_scaled[num_features] = scaler.transform(X_target[num_features])


In [20]:
# Импортируем необходимые библиотеки и модули
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# Распределение классов в тренировочных и тестовых данных
# Class distribution train: {0: 2335, 1: 1395}
# Class distribution test: {0: 2492, 1: 1571}
total = 2335 + 1395 + 1571 + 2492  # Общее количество наблюдений
count_0 = 2492 + 2335  # Общее количество наблюдений класса 0
count_1 = 1395 + 1571  # Общее количество наблюдений класса 1

# Расчет весов для классов
weight_for_0 = total / (2 * count_0)
weight_for_1 = total / (2 * count_1)

# Определение весов классов для модели
class_weights = {0: weight_for_0, 1: weight_for_1}

# Создание и настройка модели CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=150,             # Количество итераций обучения
    learning_rate=0.04,         # Скорость обучения
    depth=3,                    # Глубина деревьев
    l2_leaf_reg=4,              # Коэффициент регуляризации L2
    bootstrap_type='Bernoulli', # Тип бутстрапа
    subsample=0.8,              # Доля выборки для подвыборки
    random_seed=42,             # Случайное зерно для воспроизводимости
    eval_metric='AUC',          # Метрика оценки (ROC-AUC)
    early_stopping_rounds=100,  # Количество ранних остановок
    verbose=True,               # Вывод промежуточных результатов
    class_weights=class_weights,# Веса классов
    cat_features=['partner_src_type_ccode']  # Категориальные признаки
)

# Обучение модели на тренировочных данных
cb_model.fit(X_train_scaled, y_train)

# Предсказание вероятностей на тестовых данных
cb_y_pred = cb_model.predict_proba(X_test_scaled)[:, 1]

# Расчет метрики ROC-AUC
cb_auc = roc_auc_score(y_test, cb_y_pred)
print(f"ROC-AUC for CB: {cb_auc}")


0:	total: 4.59ms	remaining: 683ms
1:	total: 5.75ms	remaining: 425ms
2:	total: 6.41ms	remaining: 314ms
3:	total: 7.38ms	remaining: 269ms
4:	total: 8.02ms	remaining: 232ms
5:	total: 9.05ms	remaining: 217ms
6:	total: 9.69ms	remaining: 198ms
7:	total: 10.3ms	remaining: 183ms
8:	total: 10.8ms	remaining: 169ms
9:	total: 11.7ms	remaining: 163ms
10:	total: 12.3ms	remaining: 156ms
11:	total: 13ms	remaining: 149ms
12:	total: 13.7ms	remaining: 144ms
13:	total: 14.2ms	remaining: 138ms
14:	total: 14.7ms	remaining: 132ms
15:	total: 15.3ms	remaining: 128ms
16:	total: 16.4ms	remaining: 129ms
17:	total: 17ms	remaining: 125ms
18:	total: 17.5ms	remaining: 121ms
19:	total: 18.2ms	remaining: 118ms
20:	total: 19ms	remaining: 117ms
21:	total: 19.4ms	remaining: 113ms
22:	total: 20ms	remaining: 111ms
23:	total: 21ms	remaining: 110ms
24:	total: 22ms	remaining: 110ms
25:	total: 22.9ms	remaining: 109ms
26:	total: 24ms	remaining: 109ms
27:	total: 25.2ms	remaining: 110ms
28:	total: 26ms	remaining: 109ms
29:	total: 

In [21]:
# Получаем прогнозы вероятностей для целевых данных
future_predictions_unique = cb_model.predict_proba(X_target_scaled)[:, 1]

# Создаем DataFrame для финальной отправки результатов
submission_final_unique = pd.DataFrame({
    'clientbankpartner_pin': target_data['clientbankpartner_pin'],
    'score': future_predictions_unique
})

# Сохраняем результаты в CSV файл
submission_final_unique.to_csv("final_submission_unique.csv", index=False)

# Выводим первые 5 строк DataFrame для проверки
print(submission_final_unique.head())

   clientbankpartner_pin     score
0                      1  0.224942
1                      2  0.457447
2                     22  0.280730
3                     33  0.565032
4                     35  0.174391
